# <font color='gren'>Exercício 2

## **1-Imports Necessários**

In [128]:
import numpy as np
from numpy import random

import scipy
from scipy import linalg
from scipy.sparse import diags, linalg

import matplotlib.pylab as plt
from mpl_toolkits import mplot3d
from matplotlib import cm

import time

import random

## **2-Funções auxiliares**

* Função para plotar os curvas de nível constante de temperatura

In [129]:

def PlotaMembrane(N1,N2,L1,L2,W):
    x = np.linspace(0, L1, N1)
    y = np.linspace(0, L2, N2)
    X,Y=np.meshgrid(x,y)
    Z = np.copy(W)
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.set(xlabel='x', ylabel='y', title='Deslocamento vertical')
    im = ax.contourf(X, Y, Z, 20)
    im2 = ax.contour(X, Y, Z, 20, linewidths=0.25, colors='k')
    fig.colorbar(im, ax=ax)
    plt.show()
    
def PlotaSurface(N1,N2,L1,L2,W):
    x = np.linspace(0, L1, N1)
    y = np.linspace(0, L2, N2)
    X,Y=np.meshgrid(x,y)
    Z = np.copy(W)
    ax = plt.axes(projection ='3d')
    ax.plot_surface(X, Y, Z,cmap='viridis', edgecolor='none')
    ax.set(xlabel='x', ylabel='y', title='Deslocamento vertical')
    ax.set_zlim(-1.01, 1.01)
    ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,linewidth=0, antialiased=False)
    

* Montagem das matrizes K e M

In [130]:
def ij2n (i, j, N):
    return i + j*N

def BuildMatrizesEigen(N1, N2, sigma, rho, e, delta):
    nunk = N1*N2

    # Stiffness matrix K: Build it as a sparse matrix 
    d1 = 4.0*np.ones(nunk)
    d2 = -np.ones(nunk-1)
    d3 = -np.ones(nunk-N1)
    K = (sigma/delta**2)*scipy.sparse.diags([d3, d2, d1, d2, d3], [-N1, -1, 0, 1, N1], format='csr')

    # Force the eigenvalues associated to boundary points 
    # to be a big number as compared to fundamental modes
    big_number = 10000
    Iden = big_number*scipy.sparse.identity(nunk, format='csr')

    # Lados verticais
    for k in range(0,N2):
        Ic = ij2n(0,k,N1) # Left
        K[Ic,:], K[:,Ic] = Iden[Ic,:], Iden[:,Ic]

        Ic = ij2n(N1-1,k,N1) # Right
        K[Ic,:], K[:,Ic] = Iden[Ic,:], Iden[:,Ic]
        
    # Lados horizontais
    for k in range(0,N1):
        Ic = ij2n(k,0,N1) # Bottom
        K[Ic,:], K[:,Ic] = Iden[Ic,:], Iden[:,Ic]

        Ic = ij2n(k,N2-1,N1) # Top
        K[Ic,:], K[:,Ic] = Iden[Ic,:], Iden[:,Ic]

    # Mass matrix: Simple case, multiple of identity
    M = rho*e*scipy.sparse.identity(nunk, format='csr')
    
    return K, M    

## **3-Método Iterativo de Francis**

In [131]:
#   Função que encontra o maior elemento da matriz fora sua diagonal (i!=j)
def return_error(M):
    m=M
    np.fill_diagonal(m,0)
    m=np.absolute(m)
    e=m.max()
    return e

#   Função do método iterativo de Francis que retorna a matriz D de autovalores na diagonal e a matriz de autovetores Q
#com base em um erro ou um número máximo de iterações
def francis_metod(A,tol,T_max):
    Dold=A #Primeiro chute: D=A
    Dnew=Dold
    Vold=np.identity(len(A[0]))
    Vnew=Vold
    e,k=1,0
    while(e>tol and k<T_max):
        Q,R=scipy.linalg.qr(Dold)
        Dnew=np.matmul(R,Q)
        Vnew=np.matmul(Vold,Q)
        e=return_error(Dold) #while acabará quando o maior valor for muito próximo de zero ou as tentativas se esgotarem
        k+=1
        Dold=Dnew
        Vold=Vnew
    return Dold.diagonal(), Vold

## **4-Verificação de funcionamento**

In [132]:
#Pré-setagem
tol=10**(-8)
T_max=100000

A=np.array([[3,2,4],[2,0,2],[4,2,3]])
I=np.identity(3)

* Usando Francis

In [133]:
Lam,Q=francis_metod(A,10**(-8),10000)
print('Lam=',Lam)
print('Q=',Q)

Lam= [ 8. -1. -1.]
Q= [[-6.66666667e-01  4.47213596e-01 -5.96284794e-01]
 [-3.33333333e-01 -8.94427191e-01 -2.98142397e-01]
 [-6.66666667e-01  5.20652830e-11  7.45355992e-01]]


* Usando scipy para matrizes siméticas

In [134]:

Lam,Q=scipy.linalg.eigh(A,I)
print('Lam=',Lam)
print('Q=',Q)

Lam= [-4.         -1.46410162  5.46410162]
Q= [[ 7.07106781e-01  3.25057584e-01 -6.27963030e-01]
 [-1.11022302e-16 -8.88073834e-01 -4.59700843e-01]
 [-7.07106781e-01  3.25057584e-01 -6.27963030e-01]]


* Usando schipy para matrizes não simétricas

In [135]:
Lam,Q=scipy.linalg.eig(A,I)
print('Lam=',Lam)
print('Q=',Q)

Lam= [ 5.46410162+0.j -4.        +0.j -1.46410162+0.j]
Q= [[ 6.27963030e-01 -7.07106781e-01  3.25057584e-01]
 [ 4.59700843e-01 -8.21308123e-17 -8.88073834e-01]
 [ 6.27963030e-01  7.07106781e-01  3.25057584e-01]]


## **5-Resultados**

In [136]:
n=5

D=np.zeros(shape=(n,n))

for i in range(n):
    for j in range(n):
        if(i==j):
            D[i][j]=i+1

Q=np.random.rand(n,n)
A=(Q@D)@np.linalg.inv(Q)

Lam,Q=scipy.linalg.eig(A,np.identity(n))
print('A=',A)
print('Lam=',Lam)

A= [[-35.47706105  19.94582297   5.97086891  16.18260514  -7.77300646]
 [-56.28036522  31.13720861   9.46200089  23.68076973 -10.33916844]
 [-36.26193108  17.15135063  10.15286902  15.00995241  -6.36363643]
 [-23.95628625  14.33808241   2.38008196  12.92206445  -6.13263847]
 [-38.90187963  19.29009074   7.25731536  15.67627104  -3.73508102]]
Lam= [1.+0.j 2.+0.j 3.+0.j 4.+0.j 5.+0.j]
